In [1]:
%load_ext nb_black

ModuleNotFoundError: No module named 'nb_black'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

from configparser import ConfigParser


def config(filename="database.ini", section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}

    # Checks to see if section (postgresql) parser exists
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]

    # Returns an error if a parameter is called that is not listed in the initialization file
    else:
        raise Exception(
            "Section {0} not found in the {1} file".format(section, filename)
        )

    return db

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1419739404184059970
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4807060051728185338
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7766586164
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 17318266927426653248
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5623746503850044000
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [4]:
from configparser import ConfigParser


def config(filename="database.ini", section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}

    # Checks to see if section (postgresql) parser exists
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]

    # Returns an error if a parameter is called that is not listed in the initialization file
    else:
        raise Exception(
            "Section {0} not found in the {1} file".format(section, filename)
        )

    return db

In [5]:
import numpy as np


def create_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i + target_size])
    return np.array(data), np.array(labels)

In [9]:
import os

# os.environ["CUDA_DEVICE_ORDER"] = "GPU"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17883997993057543212
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 500456432938026688
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7766586164
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 6099859841202294235
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15790826381502644326
physical_device_desc: "device: XLA_GPU device"
]


In [11]:
import psycopg2
import pandas as pd

# from conf_db import config

import os

import tensorflow as tf

from tensorflow import keras

import matplotlib.pyplot as plt

# from get_data import create_data

# Establish a connection to the database by creating a cursor object

# Obtain the configuration parameters
params = config("database.ini", "warden_rady")
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params)
# Create a new cursor
cur = conn.cursor()

# A function that takes in a PostgreSQL query and outputs a pandas database
def create_pandas_table(sql_query, database=conn):
    table = pd.read_sql_query(sql_query, database)
    return table


# Utilize the create_pandas_table function to create a Pandas data frame
# Store the data as a variable
dataset1 = create_pandas_table(
    "SELECT time, value FROM history_int WHERE item_id = 6 ORDER BY time"
)
print(len(dataset1))
print(dataset1)

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

5320
                          time  value
0    2017-12-01 00:00:00+00:00      0
1    2017-12-01 01:00:00+00:00      9
2    2017-12-01 02:00:00+00:00     12
3    2017-12-01 03:00:00+00:00     18
4    2017-12-01 04:00:00+00:00     10
...                        ...    ...
5315 2018-07-10 11:00:00+00:00   1503
5316 2018-07-10 12:00:00+00:00   1610
5317 2018-07-10 13:00:00+00:00   1599
5318 2018-07-10 14:00:00+00:00   1779
5319 2018-07-10 15:00:00+00:00    827

[5320 rows x 2 columns]


In [12]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy

# fix random seed for reproducibility
np.random.seed(7)
# load the dataset
# dataframe = read_csv("airline-passengers.csv", usecols=[1], engine="python")
dataframe = dataset1["value"].to_frame()
dataset = dataframe.values
dataset = dataset.astype("float32")
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size : len(dataset), :]
# reshape into X=t and Y=t+1
look_back = 8
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX)
print(trainY)

[[[0.0000000e+00 5.1912095e-04 6.9216127e-04 ... 6.9216127e-04
   2.8840054e-04 1.1536021e-04]]

 [[5.1912095e-04 6.9216127e-04 1.0382419e-03 ... 2.8840054e-04
   1.1536021e-04 0.0000000e+00]]

 [[6.9216127e-04 1.0382419e-03 5.7680107e-04 ... 1.1536021e-04
   0.0000000e+00 0.0000000e+00]]

 ...

 [[3.2133588e-01 3.1833652e-01 3.2041299e-01 ... 2.5073543e-01
   2.5246581e-01 2.8459364e-01]]

 [[3.1833652e-01 3.2041299e-01 3.2052836e-01 ... 2.5246581e-01
   2.8459364e-01 2.6953915e-01]]

 [[3.2041299e-01 3.2052836e-01 3.1925941e-01 ... 2.8459364e-01
   2.6953915e-01 3.0305129e-01]]]
[0.         0.         0.         ... 0.26953915 0.3030513  0.29739863]


In [14]:
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(8, input_shape=(1, look_back)))
# model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# for i in range(100):
model.fit(trainX, trainY, epochs=200, batch_size=batch_size, verbose=2, shuffle=False)
#     model.reset_states()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3555 samples
Epoch 1/200
3555/3555 - 22s - loss: 0.0014
Epoch 2/200


In [ ]:
# make predictions
trainPredict = model.predict(trainX)
model.reset_states()
testPredict = model.predict(testX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print("Train Score: %.2f RMSE" % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print("Test Score: %.2f RMSE" % (testScore))

# id 24
# Train Score: 1030.19 RMSE
# Test Score: 1250.47 RMSE

# id 30
# Train Score: 60.00 RMSE
# Test Score: 65.91 RMSE

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back : len(trainPredict) + look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[
    len(trainPredict) + (look_back * 2) + 1 : len(dataset) - 1, :
] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.rcParams["figure.figsize"] = [150, 50]
plt.show()

In [ ]:
# shift train predictions for plotting
# trainPredictPlot = np.empty_like(dataset)
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back : len(trainPredict) + look_back, :] = trainPredict
# shift test predictions for plotting
# testPredictPlot = np.empty_like(dataset)
# testPredictPlot[:, :] = np.nan
# testPredictPlot[
#     len(trainPredict) + (look_back * 2) + 1 : len(dataset) - 1, :
# ] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
# plt.plot(trainPredictPlot)
# plt.plot(testPredictPlot)
plt.rcParams["figure.figsize"] = [150, 50]
plt.show()